# 1. 프로젝트 목표
- https://www.kaggle.com/c/competitive-data-science-predict-future-sales#evaluation 참고

- item_cnt_month 를 예측하는 것


# 2.문제

- https://www.kaggle.com/c/competitive-data-science-predict-future-sales/data 참고

-데이터는 files 폴더에 넣어뒀습니다.
  
  item_categories.csv (아이템의 카테고리와 아이템카테고리ID)
  
  items.csv(아이템이름,아이템ID,아이템 카테고리 ID )
  
  shops.csv(가게이름, 가게ID)
  
  sales_train.csv.gz(트레인 셋 - 날짜, 년-월기준 식별코드,가게이름, 아이템이름, 아이템가격,일별 판매량 )
  
  sample_submisson.csv(제출샘플, 테스트셋에서 id와 item_cnt_month의 정답)
  
  test.csv.gz(테스트셋)
  
  이렇게 구성되어있습니다.
  
  
# 3.할일
  
  - 트레인셋,테스트셋의 피쳐를 변형시키는 작업을 한 뒤 linear regression 모델을 적용시킵니다.
  
  - 팀회의를 통해 가설을 바탕으로 피쳐를 늘이거나 줄입니다.
  
  - 가설을 세우고 결과를 도출해 낸 후, 다른 가설을 세워 같은 작업을 반복합니다. 
  

3.일정

- 3/24 역할분담 토의, 분담한 역할 마감일 정하기


# 가설
- 카테고리별로 월별 판매량이 다를것이다
- 가격에 따라 판매량에 차이가 있을것이다.
- 온라인, 오프라인에 따라 판매량의 차이가 있을것이다
- 지역별로 온라인이 더 잘되는곳, 오프라인이 더 잘되는곳 등이 있을것이다

# Import data

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from googletrans import Translator 

In [4]:
df_test = ("../files/test.csv")
df_train = ("../files/sales_train_v2.csv")

df_test = pd.read_csv(df_test, sep = ",") #(214200, 3)
df_train = pd.read_csv(df_train, sep = ",") # (2935849, 6)
df_train['index'] = df_train.index
df_test['index'] = df_test.index

In [5]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index
0,02.01.2013,0,59,22154,999.00,1.0,0
1,03.01.2013,0,25,2552,899.00,1.0,1
2,05.01.2013,0,25,2552,899.00,-1.0,2
3,06.01.2013,0,25,2554,1709.05,1.0,3
4,15.01.2013,0,25,2555,1099.00,1.0,4


In [6]:
df_items = ("../files/items.csv")
df_item_categories = ("../files/item_categories.csv")
df_shops = ("../files/shops.csv")

df_items = pd.read_csv(df_items, sep = ",") #(214200, 3)
df_item_categories = pd.read_csv(df_item_categories, sep = ",") # (2935849, 6) 
df_shops = pd.read_csv(df_shops, sep = ",") #(214200, 3)

In [7]:
df_item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [8]:
df_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


# Translate

- 물건 카테고리이름, 상점이름등은 중요한 피쳐라 생각해 해석한 뒤 전처리하였다.


- 아이템명은 중복을 제외하고 약 20000개정도의 값을 가짐. 불필요한 정보라 생각해 드롭하였다. 


- 데이터프레임끼리 merge 하는것이 편집하기 편하기 때문에 아이템명은 드롭하지 않은 상태.



In [9]:
translator = Translator()
item_category_name_ko = pd.Series([str(translator.translate(category_name, dest = "ko").text) for category_name in df_item_categories.item_category_name])
df_item_categories.item_category_name = item_category_name_ko
df_item_categories.item_category_name


shop_name_ko = pd.Series([str(translator.translate(shop_nm, dest = "ko").text) for shop_nm in df_shops.shop_name])
df_shops.shop_name = shop_name_ko
df_shops.shop_name

0                        Yakutsk Ordzhonikidze, 56 프랑
1                                     야쿠 츠크 TC "중앙"프랑
2                                  아디 게아 쇼핑 센터 "Mega"
3                              발라 시카 TRK "10 월 키노 미르"
4            볼츠키 (Volzhsky) 쇼핑 센터 "볼가 몰 (Volga Mall)"
5                       볼 로그 다 쇼핑 및 엔터테인먼트 센터 "마멀레이드"
6                          보로 네즈 (Porkhanovskaya, 13)
7                                Voronezh TRC "막시 미르"
8                            Voronezh TRC 도시 공원 "졸업생"
9                                               외환 거래
10                          Zhukovsky Str. 차 칼로프 39m?
11                       Zhukovsky Str. Chkalov 39 m²
12                                       온라인 상점 비상 사태
13                                    카잔 TC "Bahetle"
14                                   카잔 TC "파크 하우스"II
15                                   칼루가 TRC "XXI 세기"
16                                          콜로세움 "리오"
17    크라스 노야 르 스크 (Krasnoyarsk) 쇼핑 센터 "Vzletka Plaza"
18             크라스 노야 르 스크 (

In [10]:
# -*- coding: utf-8 -*-
def split_words(df_col_values):
    tokenized = [i.split(" ") for i in df_col_values]
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') 
    
    for i in range(len(df_col_values)):
        df_col_values[i] = tokenized[i][0] if hangul.sub('',tokenized[i][0]) else "기타"
    return df_col_values

split_words(df_item_categories.item_category_name.values)
split_words(df_shops.shop_name.values)

array(['기타', '야쿠', '아디', '발라', '볼츠키', '볼', '보로', '기타', '기타', '외환', '기타',
       '기타', '온라인', '카잔', '카잔', '칼루가', '콜로세움', '크라스', '크라스', '쿠르스크',
       '모스크바', '모스크바', '모스크바', '모스크바', '모스크바', '모스크바', '모스크바', '모스크바',
       '모스크바', '모스크바', '모스크바', '모스크바', '모스크바', '기타', '기타', '니즈니',
       '노보시비르스크', '노보시비르스크', '옴', '기타', '기타', '기타', '기타', '기타', '사마라',
       '사마라', '세르', '기타', '톰', '기타', '튜멘', '튜멘', '우파', '우파', '기타', '디지털',
       '체', '기타', '야쿠', '야로'], dtype=object)

# Simplify Categories 
- 임시임, 수정해야함

In [23]:
def rebuild_id(id_name,df,str_column) : 
    setted = {j:i for i,j in (enumerate(set(df[str_column])))}
    kor_values = df.set_index(id_name)[str_column].to_dict().values()

    get_numeric_values = lambda x : setted[x]
    numeric_values = list(map(get_numeric_values,kor_values))
    numeric_dict = dict(enumerate(numeric_values))

    for key in numeric_dict.keys(): 
        df[id_name][key] = numeric_dict[key]
    
    return df

rebuild_id("shop_id",df_shops,"shop_name")

/home/hyuna/anaconda3/envs/deep_learning_study/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


,item_category_name,item_category_id,item_id
0,기타,16,5441
1,기타,12,16255
2,기타,12,16256
3,기타,12,16257
4,액세서리,12,5606
5,액세서리,12,5740
6,액세서리,12,5570
7,액세서리,12,5571
8,액세서리,3,5572
9,액세서리,13,5573


# Merge & Drop

- 이제 정제된 데이터들만 갖고가기 위해 Merge, Drop 해 주었다.


In [12]:
#아이템카테고리 id만 df_train,df_test에 존재 -> items + item_categories -> df_train + item_info
df_item_info = pd.merge(df_item_categories, df_items, on = "item_category_id")
df_item_info.head(5)

,item_category_name,item_category_id,item_name,item_id
0,기타,0,PC: Гарнитура HyperX Cloud Core игровая стерео...,5441
1,기타,0,Наушники PHILIPS SBC HC8680,16255
2,기타,0,Наушники RITMIX RH-120,16256
3,기타,0,Наушники RITMIX RH-124 Black,16257
4,액세서리,1,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606


In [13]:
del df_item_info["item_name"] #item_name의 데이터는 너무 방대함 -> 카테고리로 충분하다 판단

In [14]:
df_item_info.head(5)

,item_category_name,item_category_id,item_id
0,기타,0,5441
1,기타,0,16255
2,기타,0,16256
3,기타,0,16257
4,액세서리,1,5606


In [15]:
df_train = pd.merge(df_train,df_item_info ,on = "item_id")
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_name,item_category_id
0,02.01.2013,0,59,22154,999.0,1.0,0,시네마,37
1,23.01.2013,0,24,22154,999.0,1.0,3270,시네마,37
2,20.01.2013,0,27,22154,999.0,1.0,17081,시네마,37
3,02.01.2013,0,25,22154,999.0,1.0,25918,시네마,37
4,03.01.2013,0,25,22154,999.0,1.0,25919,시네마,37


In [16]:
df_test = pd.merge(df_test,df_item_info,on = "item_id")
df_test.head()

,ID,shop_id,item_id,index,item_category_name,item_category_id
0,0,5,5037,0,게임,19
1,5100,4,5037,5100,게임,19
2,10200,6,5037,10200,게임,19
3,15300,3,5037,15300,게임,19
4,20400,2,5037,20400,게임,19


In [17]:
df_train = pd.merge(df_train, df_shops, on = "shop_id")
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_name,item_category_id,shop_name
0,23.01.2013,0,24,22154,999.0,1.0,3270,시네마,37,발라
1,10.10.2015,33,24,22154,299.0,1.0,2930033,시네마,37,발라
2,01.02.2013,1,24,2573,249.0,1.0,153171,음악,55,발라
3,17.01.2013,0,24,2574,399.0,1.0,2807,음악,55,발라
4,01.09.2013,8,24,2574,399.0,1.0,865971,음악,55,발라


In [18]:
df_test = pd.merge(df_test, df_shops, on = "shop_id")
df_test.head()

,ID,shop_id,item_id,index,item_category_name,item_category_id,shop_name
0,0,5,5037,0,게임,19,아디
1,1,5,5320,1,음악,55,아디
2,2,5,5233,2,게임,19,아디
3,3,5,5232,3,게임,23,아디
4,4,5,5268,4,게임,20,아디


In [19]:
df_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_name,item_category_id,shop_name
0,23.01.2013,0,24,22154,999.0,1.0,3270,시네마,37,발라
1,10.10.2015,33,24,22154,299.0,1.0,2930033,시네마,37,발라
2,01.02.2013,1,24,2573,249.0,1.0,153171,음악,55,발라
3,17.01.2013,0,24,2574,399.0,1.0,2807,음악,55,발라
4,01.09.2013,8,24,2574,399.0,1.0,865971,음악,55,발라


#  가격 변동여부 적용

- 가격정보는 명목형 값이므로 최소화해야한다는 생각을 하였다.


- 한 아이템내의 가격을 sort 한 뒤 오름차순으로 등수를 매겼다.



- 딕트타입으로 row 의 인덱스값을 키값, 등수를 매긴 가격정보를 밸류값으로 저장한 뒤 데이터프레임 생성

In [20]:
def rank_per_item_price(df):
    
    df_grouped = df.groupby(["shop_id","item_category_id","item_id"])["item_price"].apply(list)
    #같은 제품에 있는 가격들을 list로 묶음
    ranked_price_mat = []
    for price_list in df_grouped.values:
        ranked_price_dict = {}
        ranked_price = {price : no  for no,price in enumerate(sorted(np.unique(price_list)))}
        ranked_price_dict.update(ranked_price)
        ranked_price_mat.append(ranked_price_dict)
    
    # 가격에 따라 enumerate로 등수를 매김, 현재 가격과 등수가 각각 dict값의 키와 밸류로 저장되어있음

    for value_no in range(len(df_grouped.values)): 
        for index in range(len(df_grouped.values[value_no])):
            df_grouped.values[value_no][index] = ranked_price_mat[value_no][df_grouped.values[value_no][index]]
        
    #df_grouped에 변경사항 반영

    df_get_back = df.groupby(["shop_id","item_category_id","item_id"])["index"].apply(list)
    get_back_dict = {}
    for value_no in range(len(df_grouped.values)): 
        for index in range(len(df_grouped.values[value_no])):
            get_back_dict.update({df_get_back.values[value_no][index] : df_grouped.values[value_no][index]+1})
        
    #item_price를 원래대로 돌리기 위해 index값을 똑같이 묶은 뒤 딕트타입으로 키값에 인덱스번호를 부여
    #그리고 enumerate한 등수가 0이되지 않도록 1을 더해줌. {인덱스값 : 가격} 형태의 딕트 완성
        
    df_ranked = pd.DataFrame(list(get_back_dict.items()),columns=['index','new_item_price'])
    df = pd.merge(df,df_ranked, on = "index")
    return df

In [21]:
rank_per_item_price(df_train)

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,index,item_category_name,item_category_id,shop_name,new_item_price
0,23.01.2013,0,24,22154,999.0,1.0,3270,시네마,37,발라,2
1,10.10.2015,33,24,22154,299.0,1.0,2930033,시네마,37,발라,1
2,01.02.2013,1,24,2573,249.0,1.0,153171,음악,55,발라,1
3,17.01.2013,0,24,2574,399.0,1.0,2807,음악,55,발라,1
4,01.09.2013,8,24,2574,399.0,1.0,865971,음악,55,발라,1
5,22.10.2013,9,24,2574,399.0,2.0,982913,음악,55,발라,1
6,22.12.2013,11,24,2574,399.0,1.0,1136962,음악,55,발라,1
7,15.01.2014,12,24,2574,399.0,1.0,1366484,음악,55,발라,1
8,18.04.2014,15,24,2574,399.0,1.0,1570727,음악,55,발라,1
9,20.05.2014,16,24,2574,399.0,1.0,1660503,음악,55,발라,1


In [22]:
del df_train["date"]
del df_train["index"]
del df_train["item_price"]
del df_train["shop_id"]
del df_train["item_id"]
df_train.head(5)

,date_block_num,item_cnt_day,item_category_name,item_category_id,shop_name
0,0,1.0,시네마,37,발라
1,33,1.0,시네마,37,발라
2,1,1.0,음악,55,발라
3,0,1.0,음악,55,발라
4,8,1.0,음악,55,발라


In [13]:
# df_grouped.rank(pct = True) #아이템 가격별로 랭크매기기